In [1]:
import helpers as helpers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/griffinmichalak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# TfidfVectorizer(max_features=50k, ngram_range=(1,2), min_df=2) 

DEPRESSION_PATH = "data/depression_dataset_reddit_cleaned.csv"
TRAIN_DEV_TEST = "80/10/10"

In [3]:
data = helpers.read_file(datapath=DEPRESSION_PATH, ngram=2, tokenize=False, by_character=False)
clean_text = []
is_depression = []
for row in data:
    clean_text.append(row[0])
    is_depression.append(row[1])

In [4]:
train_text, dev_text, test_text = helpers.split(data=clean_text, dist=TRAIN_DEV_TEST)
train_dep, dev_dep, test_dep = helpers.split(data=is_depression, dist=TRAIN_DEV_TEST)

Completing 80/10/10 split
Completing 80/10/10 split


In [5]:
# build TfidfVectorizer
model = TfidfVectorizer(max_features=50_000, ngram_range=(1,2), min_df=2)
model.fit(train_text)
train_vectors = model.transform(train_text)
dev_vectors = model.transform(dev_text)
test_vectors = model.transform(test_text)

In [6]:
weight = None
# class_weight='balanced'
lr = LogisticRegression(C = 1.0, max_iter=2000, class_weight=weight, solver='liblinear')
lr.fit(train_vectors, train_dep)

dev_predictions = lr.predict(dev_vectors)

In [7]:
precision = precision_score(dev_dep, dev_predictions)
recall = recall_score(dev_dep, dev_predictions)
f1 = f1_score(dev_dep, dev_predictions)
accuracy = accuracy_score(dev_dep, dev_predictions)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")
print(f"Accuracy: {accuracy}")

Precision: 0.975609756097561
Recall: 0.9183673469387755
F1: 0.9461235216819974
Accuracy: 0.9469598965071151
